## Notebook to process the ouput of deepcell and generated segmented cells

In [1]:
import numpy as np
import os
import skimage.io as io
import matplotlib.pyplot as plt
import sys

sys.path.append("../")

from segmentation.utils import data_utils, segmentation_utils
import xarray as xr

In [2]:
base_dir = '../data/example_dataset/'

# base name of network that will be used to save intermediate files
base_name = "deepcell_output"

deepcell_dir = os.path.join(base_dir, base_name)

In [3]:
# save deepcell_outputs as TIFs, transform and smooth for watershed
pixel_xr_data = xr.open_dataarray(os.path.join(deepcell_dir, base_name + "_pixel.nc"))
pixel_xr_data.name = base_name + "_pixel"
data_utils.save_deepcell_tifs(pixel_xr_data, save_path=deepcell_dir,  transform='pixel', pixel_smooth=[5])

In [4]:
segmentation_dir = base_dir + "/segmentation_output/"
if not os.path.isdir(segmentation_dir):
    os.makedirs(segmentation_dir)

In [6]:
# load the processed deepcell output
pixel_xr = xr.open_dataarray(os.path.join(deepcell_dir, '{}_pixel_processed.nc'.format(base_name)))
input_xr = xr.open_dataarray(os.path.join(base_dir, "input_data/deepcell_input.nc"))

# watershed over the processed deepcell output
segmentation_dir = base_dir + "/segmentation_output/"
if not os.path.isdir(segmentation_dir):
    os.makedirs(segmentation_dir)

segmentation_utils.watershed_transform(pixel_xr=pixel_xr, channel_xr=input_xr, background_threshold=0.35,
                                       watershed_maxs=False, pixel_smooth="pixel_interior_smoothed_5",
                                       overlay_channels=["HH3", "Membrane"], output_dir=segmentation_dir, 
                                       rescale_factor=1.5)



analyzing point Point8


In [7]:
# load segmentation generated by watershed
segmentation_labels = xr.open_dataarray(os.path.join(segmentation_dir,
                                                     '{}_pixel_processed_segmentation_labels.nc'.format(base_name)))


In [8]:
# load channel data
points_folder = os.path.join(base_dir, "Input_Data")
tif_folder = 'TIFs'
points = os.listdir(points_folder)
points = [point for point in points if os.path.isdir(os.path.join(points_folder, point))]
image_data = data_utils.load_tifs_from_points_dir(point_dir=points_folder, tif_folder=tif_folder, points=points)


In [9]:
single_cell_dir = base_dir + "single_cell_output/"

if not os.path.exists(single_cell_dir):
    os.makedirs(single_cell_dir)

# segment the imaging data
segmentation_utils.extract_single_cell_data(segmentation_labels=segmentation_labels, image_data=image_data,
                                          save_dir=single_cell_dir)

# combine CSV files together
csv_files = os.listdir(single_cell_dir)
csv_files = [x for x in csv_files if 'transformed' in x]

segmentation_utils.concatenate_csv(base_dir=single_cell_dir, csv_files=csv_files)

extracting data from Point8
